In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import amplpy
from pathlib import Path
import os
import shutil

from util.return_up_outliers_func import *

In [2]:
# Path for the run
path_unit_tests = Path(r'..\..\unit_tests')
path_data_network = Path(r'.\data_network')
path_ampl_results = Path(r'.\results')

In [3]:
# Import data
#df_branches = pd.read_table(path_data_network / 'ampl_network_branches.txt', sep=" ", header=1).drop(columns=["#\"variant\""])
#df_buses = pd.read_table(path_data_network / 'ampl_network_buses.txt', sep=" ", header=1).drop(columns=["#\"variant\""])

In [4]:
# TODO : test the following function

def eval_level_of_correctness_buses(df_buses, expected, results, verbose=False):
    """ Eval the level of detection for buses in opti results.
    
    df_buses : pandas.dataframe containing network buses information.
    expected : the substation expected.
    results : the buses detected with an error.
    
    If one the buses of results is in the substation expected, return 1. Otherwise, return -1.
    """

    df_substation = pd.read_table(path_data_network / 'ampl_network_substations.txt', sep=" ", header=1).drop(columns=["#\"variant\""])
    num_substation_expected = df_substation[df_substation["id"] == expected]["num"].to_numpy()[0]

    for bus in results:
        if df_buses[df_buses["id"] == bus]["substation"].to_numpy()[0] == num_substation_expected :
            return 1

    return -1

def eval_level_of_correctness_branches(df_branches, expected, results, verbose=False):
    """ Eval the level of detection for branches in opti results. 

    df_branches : pandas.dataframe containing information on branches of the network.
    expected : the branche expected.
    results : the branches detected with an error.

    If expected is in results, return 1.
    Else if expected share a common bus with one branch of results, return 0.
    Else, return -1."""
    
    if expected in results:
        return 1
    
    elif len(df_branches[df_branches['id'] == expected]['bus1'].to_numpy()) != 0 and len(df_branches[df_branches['id'] == expected]['bus2'].to_numpy()) != 0:
        bus1_exp = df_branches[df_branches['id'] == expected]['bus1'].to_numpy()[0]
        bus2_exp = df_branches[df_branches['id'] == expected]['bus2'].to_numpy()[0]

        for result in results:

            buses_result = [df_branches[df_branches['id'] == result]['bus1'].to_numpy()[0], df_branches[df_branches['id'] == result]['bus2'].to_numpy()[0]]
            if bus1_exp in buses_result or bus2_exp in buses_result:
                return 0 # FIXME ICI BIZARRE QU'ON AIT JAMAIS 0.

    return -1

def get_penalized_buses(df_results, threshold):
    """ Return the buses that are penalized. A penalized bus is a bus with a high sigma value on it."""
    buses_detected = set()
    df_data = pd.read_table(path_data_network / 'ampl_network_buses.txt', sep=" ", header=1).drop(columns=["#\"variant\""])

    for index, _ in return_up_outliers_Zscore(df_results["s1"].abs(), threshold).items():
        buses_detected.add(df_data['id'].get(df_results['busPV'].get(index) - 1))

    return buses_detected

def get_penalized_branches(df_results, threshold):
    """ Return the branches that are penalized. A penalized branch is a branch with a high sigma value on it."""

    branches_detected = set()
    df_data = pd.read_table(path_data_network / 'ampl_network_branches.txt', sep=" ", header=1).drop(columns=["#\"variant\""])
    
    for i in range(1,9):
        for index, _ in return_up_outliers_Zscore(df_results["sigma"+str(i)].abs(), threshold).items():
            branches_detected.add(df_data['id'].get(df_results['branch'].get(index) - 1))

    return branches_detected

df_results = pd.read_csv(path_ampl_results / "sigma_penal_dbb.csv", header=1, sep=";")
print(get_penalized_branches(df_results, 3))

#df_data = pd.read_table(path_data_network / 'ampl_network_branches.txt', sep=" ", header=1).drop(columns=["#\"variant\""])
#print(get_penalized_branches(pd.read_csv(path_ampl_results / "sigma_penal_dbb.csv", header=1, sep=";"), 3))
#print(df_data[df_data['id'] == "AVELIY762"])

# Tests
#df_data = pd.read_table(path_data_network / 'ampl_network_branches.txt', sep=" ", header=1).drop(columns=["#\"variant\""])
#expected = "1f1ef22c-9735-4e48-908c-0c735f985a66"
#
#print("Expected result : 1")
#print("Result :", eval_level_of_correctness(df_data, expected, {expected}))
#print("Expected result : 1")
#print("Result :", eval_level_of_correctness(df_data, expected, {expected, "557b1e64-5645-484b-991b-377f8fd6512b"}))
#print("Expected result : 0")
#print("Result :", eval_level_of_correctness(df_data, expected, {"420c3d64-124e-4465-8616-1d4463fd5b8b"}))
#print("Expected result : 0")
#print("Result :", eval_level_of_correctness(df_data, expected, {"420c3d64-124e-4465-8616-1d4463fd5b8b", "557b1e64-5645-484b-991b-377f8fd6512b"}))
#print("Expected result : 0")
#print("Result :", eval_level_of_correctness(df_data, expected, {"3e94dffe-20cd-4322-a38b-111d277dae98"}))
#print("Expected result : 0")
#print("Result :", eval_level_of_correctness(df_data, expected, {"557b1e64-5645-484b-991b-377f8fd6512b", "3e94dffe-20cd-4322-a38b-111d277dae98"}))
#print("Expected result : -1")
#print("Result :", eval_level_of_correctness(df_data, expected, {"557b1e64-5645-484b-991b-377f8fd6512b"}))


{'.NIEDL71.UCHT', '.TDVAL72.VANY'}


In [5]:
threshold = 3

expected = {}
results = {}
scores = {}

for test_category in os.listdir(path_unit_tests):
    expected[test_category] = {}
    results[test_category] = {}
    scores[test_category] = {}

    for unit_test in os.listdir(path_unit_tests / str(test_category)):

        path_unit_test = Path(path_unit_tests /test_category / unit_test)

        # filter tests with no expected results
        if "results_expected.txt" not in os.listdir(path_unit_test):
            continue

        # Here we know that there is 2 lines because we expect only one branch with an error
        with open(path_unit_test / "results_expected.txt", 'r') as file:
            file.readline() # Skip the comment on first line
            expected[test_category][unit_test] = file.readline().replace('\n', '')

        # Copy the network files in corresponding directory
        for file in os.listdir(path_unit_test):
            shutil.copyfile(path_unit_test / file, path_data_network / file)

        # Execute the ampl file corresponding to error detection model
        ampl = amplpy.AMPL()
        ampl.read('detection.run') # So we export results files in path_ampl_results
        ampl.close()

        # Read results after running

        if test_category == "targetV":
            df_results_s = pd.read_csv(path_ampl_results / "s1_penal.csv", header=1, sep=";") # TODO : Be careful here, path is absolute
            df_buses = pd.read_table(path_data_network / 'ampl_network_buses.txt', sep=" ", header=1).drop(columns=["#\"variant\""])
            results[test_category][unit_test] = get_penalized_buses(df_results_s, threshold)
            scores[test_category][unit_test] = eval_level_of_correctness_buses(df_results_s, expected[test_category][unit_test], results[test_category][unit_test]) 
        else:
            df_results_sigma = pd.read_csv(path_ampl_results / "sigma_penal_dbp.csv", header=1, sep=";") # TODO : Be careful here, path is absolute
            results[test_category][unit_test] = get_penalized_branches(df_results_sigma, threshold)
            scores[test_category][unit_test] = eval_level_of_correctness_branches(df_results_sigma, expected[test_category][unit_test], results[test_category][unit_test])
        
        print("All expected = ", expected)
        print("All results =", results)

        print("For test cat " + test_category + ", " + unit_test + " : ")
        print("Score of current results : ", scores[test_category][unit_test])
        print("Number of elements in current results : ", len(results[test_category][unit_test]))






















*** Start of file divergence analysis : Fri Apr 28 14:38:18 2023
Parameter: threshold to decide wether an active or reactive power value is zero Pnull:=0.01 (MW or Mvar or MVA)
Parameter: threshold to detect zero impedance branch Znull:=0.0001 pu
Parameter: for consistency checks of minimum nominal voltages epsilon_nominal_voltage:= 1 kV
Parameter: for consistency checks of voltage bounds eps<=Vmin<Vmax<=2-eps, epsilon_min_voltage:= 0.5 pu
Parameter: maximum for generating units parameters Pmin Pmax Qmin Qmax = 9000 MW or Mvar
Parameter: defaultPmax = 1000 MW
Parameter: defaultPmin = 0 MW
Parameter: defaultQmaxPmaxRatio = 0.3 Mvar/MW
Parameter: defaultQmin = -300 Mvar
Parameter: defaultQmax = 300 Mvar
Parameter: minimalQPrange = 1 MW or Mvar

*** Connexity computation
# CCcomp solve: start (Fri Apr 28 14:38:18 2023)

Artelys Knitro 13.2.0: outlev=0
##### This license is only intended for use by Artelys. #####
##### License is valid until Jan 31, 2024 #####
Knitro 13

RuntimeError: Unknown exception

In [ ]:
global_scores = {}
for param in scores.keys():
    global_scores[param] = 0

    for test in scores[param].keys():
        print("Result for" + " " + param + "test " + test)
        print(results[param][test]) 
        if scores[param][test] > -1:
            global_scores[param] += 1

        #print("For param " + param + " and test " + test + ", score = " + str(scores[param][test]))
        #print("Expected branches was : " + expected[param][test])
        #print("Results gotten : " + str(results[param][test]) + "\n")

# Resume
for param in global_scores.keys():
    print("For parameter " + param + " :")
    print("Number of tests passed : " + str(global_scores[param]) + " / " + str(len(results[param])))
    print("Number of tests wrong : " + str(len(results[param]) - global_scores[param]) + " / " + str(len(results[param])) + "\n")

print(scores)

Result for alphatest test0
{'.TDVAL72.VANY'}
Result for alphatest test1
{'LESTRY642', 'SUPP2L71SUPP3', '.TDVAL72.VANY'}
Result for alphatest test10
{'.NIEDL71.UCHT'}
Result for alphatest test11
{'.TDVAL72.VANY'}
Result for alphatest test2
{'.DAHLL71.OBER', '.NIEDL71.UCHT', '.DAHLL72.NIED'}
Result for alphatest test3
{'.DAHLL72.OBER', 'M.TERY411', '.NIEDL71.UCHT', '.DAHLL72.NIED', '.DAHLL71.OBER', '.DAHLL71.NIED'}
Result for alphatest test4
{'.NIEDL71.UCHT'}
Result for alphatest test5
{'.TDVAL72.VANY'}
Result for alphatest test6
{'.NIEDL71.UCHT'}
Result for alphatest test7
{'.TDVAL72.VANY'}
Result for alphatest test8
{'.NIEDL71.UCHT'}
Result for alphatest test9
{'.DAHLL72.OBER', '.DAHLL71.OBER', '.NIEDL71.UCHT', '.DAHLL72.NIED'}
Result for B1test test0
{'BILEMY411', 'HELLEL61ZHEL5', 'M.TERY411', 'AVELIY761'}
Result for B1test test1
{'AIRAIL41CROI5', 'ARGOEL61BLOCA', 'BLOCAL61LIMEU', 'BLOCAL41CROI5', 'BLOCAY643', 'ARGOEY771', 'B.CO5Y611', 'AIRAIL41ARGOE', 'ARGOEY641', 'LIMEUY771'}
Result